# Deploying Iris-detection model using Vertex AI


## Overview

In this tutorial, you build a scikit-learn model and deploy it on Vertex AI using the custom container method. You use the FastAPI Python web server framework to create a prediction endpoint. You also incorporate a preprocessor from training pipeline into your online serving application.

Learn more about [Custom training](https://cloud.google.com/vertex-ai/docs/training/custom-training) and [Vertex AI Prediction](https://cloud.google.com/vertex-ai/docs/predictions/get-predictions).

### Objective

In this notebook, you learn how to create, deploy and serve a custom classification model on Vertex AI. This notebook focuses more on deploying the model than on the design of the model itself. 


This tutorial uses the following Vertex AI services and resources:

- Vertex AI models
- Vertex AI endpoints

The steps performed include:

- Train a model that uses flower's measurements as input to predict the class of iris.
- Save the model and its serialized pre-processor.
- Build a FastAPI server to handle predictions and health checks.
- Build a custom container with model artifacts.
- Upload and deploy custom container to Vertex AI Endpoints.

### Dataset

This tutorial uses R.A. Fisher's Iris dataset, a small and popular dataset for machine learning experiments. Each instance has four numerical features, which are different measurements of a flower, and a target label that
categorizes the flower into: **Iris setosa**, **Iris versicolour** and **Iris virginica**.

This tutorial uses [a version of the Iris dataset available in the
scikit-learn library](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_iris.html#sklearn.datasets.load_iris).

### Costs 

This tutorial uses billable components of Google Cloud:

* Vertex AI
* Cloud Storage
* Artifact Registry
* Cloud Build

Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), [Artifact Registry pricing](https://cloud.google.com/artifact-registry/pricing) and [Cloud Build pricing](https://cloud.google.com/build/pricing) and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages



In [1]:

# Vertex SDK for Python
! pip3 install --upgrade --quiet  google-cloud-aiplatform

### Set Google Cloud project information 
Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [2]:
PROJECT_ID = "neon-gist-461209-a0"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [3]:
BUCKET_URI = f"gs://mlops-course-neon-gist-461209-a0-unique"  # @param {type:"string"}

**If your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [4]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

Creating gs://mlops-course-neon-gist-461209-a0-unique/...
ServiceException: 409 A Cloud Storage bucket named 'mlops-course-neon-gist-461209-a0-unique' already exists. Try another name. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.


### Initialize Vertex AI SDK for Python

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com). 

In [5]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Import the required libraries

In [6]:
import os
import sys

### Configure resource names

Set a name for the following parameters:

`MODEL_ARTIFACT_DIR` - Folder directory path to your model artifacts within a Cloud Storage bucket, for example: "my-models/fraud-detection/trial-4"

`REPOSITORY` - Name of the Artifact Repository to create or use.

`IMAGE` - Name of the container image that is pushed to the repository.

`MODEL_DISPLAY_NAME` - Display name of Vertex AI model resource.

In [7]:
MODEL_ARTIFACT_DIR = "my-models/iris-classifier-week-1"  # @param {type:"string"}
REPOSITORY = "iris-classifier-repo"  # @param {type:"string"}
IMAGE = "iris-classifier-img"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "iris-classifier"  # @param {type:"string"}

# Set the defaults if no names were specified
if MODEL_ARTIFACT_DIR == "[your-artifact-directory]":
    MODEL_ARTIFACT_DIR = "custom-container-prediction-model"

if REPOSITORY == "[your-repository-name]":
    REPOSITORY = "custom-container-prediction"

if IMAGE == "[your-image-name]":
    IMAGE = "sklearn-fastapi-server"

if MODEL_DISPLAY_NAME == "[your-model-display-name]":
    MODEL_DISPLAY_NAME = "sklearn-custom-container"

## Simple Decision Tree model
Build a Decision Tree model on iris data

In [8]:
import mlflow
from mlflow import MlflowClient
from mlflow.models import infer_signature
from pprint import pprint

mlflow.set_tracking_uri("http://127.0.0.1:8100")
client = MlflowClient(mlflow.get_tracking_uri())
all_experiments = client.search_experiments()
print(all_experiments)

[<Experiment: artifact_location='mlflow-artifacts:/944550546737150720', creation_time=1751626953154, experiment_id='944550546737150720', last_update_time=1751626953154, lifecycle_stage='active', name='IRIS Classifier : Mflow demo', tags={}>, <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1751623248067, experiment_id='0', last_update_time=1751623248067, lifecycle_stage='active', name='Default', tags={}>]


In [9]:
mlflow.get_tracking_uri()

'http://127.0.0.1:8100'

In [10]:
mlflow.set_experiment("IRIS Classifier : Mflow demo")

<Experiment: artifact_location='mlflow-artifacts:/944550546737150720', creation_time=1751626953154, experiment_id='944550546737150720', last_update_time=1751626953154, lifecycle_stage='active', name='IRIS Classifier : Mflow demo', tags={}>

In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from pandas.plotting import parallel_coordinates
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn import metrics

data = pd.read_csv('data/iris.csv')
data.head(5)

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [12]:
train, test = train_test_split(data, test_size = 0.4, stratify = data['species'], random_state = 42)
X_train = train[['sepal_length','sepal_width','petal_length','petal_width']]
y_train = train.species
X_test = test[['sepal_length','sepal_width','petal_length','petal_width']]
y_test = test.species

In [13]:
params = {
    "max_depth":4,
    "random_state":2,
    "min_samples_split": 2,
    "min_samples_leaf": 1,
}

mod_dt = DecisionTreeClassifier(**params)
mod_dt.fit(X_train,y_train)
prediction=mod_dt.predict(X_test)
accuracy_score = metrics.accuracy_score(prediction,y_test)
print('The accuracy of the Decision Tree is',"{:.3f}".format(metrics.accuracy_score(prediction,y_test)))

The accuracy of the Decision Tree is 0.950


In [14]:
import pickle
import joblib

joblib.dump(mod_dt, "artifacts/model.joblib")

['artifacts/model.joblib']

### Upload model artifacts and custom code to Cloud Storage

Before you can deploy your model for serving, Vertex AI needs access to the following files in Cloud Storage:

* `model.joblib` (model artifact)
* `preprocessor.pkl` (model artifact)

Run the following commands to upload your files:

In [15]:
!gsutil cp artifacts/model.joblib {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

Copying file://artifacts/model.joblib [Content-Type=application/octet-stream]...
/ [1 files][  2.9 KiB/  2.9 KiB]                                                
Operation completed over 1 objects/2.9 KiB.                                      


In [16]:
with mlflow.start_run():
    mlflow.log_params(params)
    mlflow.log_metric("accuracy",accuracy_score)
    mlflow.set_tag("Training Info", "Decision tree model for IRIS")
    signature = infer_signature(X_train, mod_dt.predict(X_train))
    
    model_info = mlflow.sklearn.log_model(
        sk_model = mod_dt,
        artifact_path = "iris_model",
        signature = signature,
        input_example = X_train,
        registered_model_name = "IRIS-classifier-dt"
    )

2025/07/05 05:45:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Registered model 'IRIS-classifier-dt' already exists. Creating a new version of this model...
2025/07/05 05:45:57 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt, version 5


🏃 View run smiling-bird-781 at: http://127.0.0.1:8100/#/experiments/944550546737150720/runs/de7666a4e9bb4184a9bc1bb0ca9e09a4
🧪 View experiment at: http://127.0.0.1:8100/#/experiments/944550546737150720


Created version '5' of model 'IRIS-classifier-dt'.


## Hyperparameter Tuning

In [17]:
mlflow.set_experiment("IRIS Classifier: GridSearchCV All Runs")


2025/07/05 05:46:27 INFO mlflow.tracking.fluent: Experiment with name 'IRIS Classifier: GridSearchCV All Runs' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/384316239563522829', creation_time=1751694387367, experiment_id='384316239563522829', last_update_time=1751694387367, lifecycle_stage='active', name='IRIS Classifier: GridSearchCV All Runs', tags={}>

In [18]:
model = DecisionTreeClassifier(random_state=42)
param_grid = {
    "max_depth": [2, 3, 4],
    "min_samples_split": [2, 5],
    "min_samples_leaf": [1, 2]
}

In [20]:
from sklearn.model_selection import train_test_split, GridSearchCV

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, return_train_score=True)
grid_search.fit(X_train, y_train)

,estimator,DecisionTreeC...ndom_state=42)
,param_grid,"{'max_depth': [2, 3, ...], 'min_samples_leaf': [1, 2], 'min_samples_split': [2, 5]}"
,scoring,None
,n_jobs,None
,refit,True
,cv,3
,verbose,0
,pre_dispatch,'2*n_jobs'
,error_score,nan
,return_train_score,True
,criterion,'gini'


In [25]:
for i in range(len(grid_search.cv_results_["params"])):
    params = grid_search.cv_results_["params"][i]
    mean_test_score = grid_search.cv_results_["mean_test_score"][i]
    std_test_score = grid_search.cv_results_["std_test_score"][i]

    model_i = DecisionTreeClassifier(**params, random_state=42)
    model_i.fit(X_train, y_train)
    predictions = model_i.predict(X_test)
    acc_score = metrics.accuracy_score(prediction, y_test)

    
    with mlflow.start_run(run_name=f"grid_run_{i+1}"):
        mlflow.log_params(params)
        mlflow.log_metric("cv_accuracy_mean", mean_test_score)
        mlflow.log_metric("cv_accuracy_std", std_test_score)
        mlflow.log_metric("test_accuracy", acc_score)

        mlflow.set_tag("type", "gridsearch_full")
        mlflow.set_tag("cv", "3-fold")

        signature = infer_signature(X_train, model_i.predict(X_train))
        mlflow.sklearn.log_model(
            sk_model=model_i,
            artifact_path="iris_model",
            input_example=X_train,
            signature=signature,
            registered_model_name = "IRIS-classifier-dt-grid_Search"
        )

# Print best params
print("✅ Best Params:", grid_search.best_params_)
print("✅ Best CV Score:", grid_search.best_score_)

2025/07/05 06:01:48 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
Successfully registered model 'IRIS-classifier-dt-hyperparameter'.
2025/07/05 06:01:51 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt-hyperparameter, version 1
Created version '1' of model 'IRIS-classifier-dt-hyperparameter'.
2025/07/05 06:01:52 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run grid_run_1 at: http://127.0.0.1:8100/#/experiments/384316239563522829/runs/443dcb5b4deb46d1a2c0ff6d8da8a548
🧪 View experiment at: http://127.0.0.1:8100/#/experiments/384316239563522829


Registered model 'IRIS-classifier-dt-hyperparameter' already exists. Creating a new version of this model...
2025/07/05 06:01:55 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt-hyperparameter, version 2
Created version '2' of model 'IRIS-classifier-dt-hyperparameter'.
2025/07/05 06:01:55 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run grid_run_2 at: http://127.0.0.1:8100/#/experiments/384316239563522829/runs/873d37c5a2454c1792f14e4b65d56f49
🧪 View experiment at: http://127.0.0.1:8100/#/experiments/384316239563522829


Registered model 'IRIS-classifier-dt-hyperparameter' already exists. Creating a new version of this model...
2025/07/05 06:01:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt-hyperparameter, version 3
Created version '3' of model 'IRIS-classifier-dt-hyperparameter'.
2025/07/05 06:01:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


...

Registered model 'IRIS-classifier-dt-hyperparameter' already exists. Creating a new version of this model...
2025/07/05 06:02:27 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt-hyperparameter, version 11
Created version '11' of model 'IRIS-classifier-dt-hyperparameter'.
2025/07/05 06:02:27 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run grid_run_11 at: http://127.0.0.1:8100/#/experiments/384316239563522829/runs/cc508e13d70e4ad194e12d00e9920100
🧪 View experiment at: http://127.0.0.1:8100/#/experiments/384316239563522829


Registered model 'IRIS-classifier-dt-hyperparameter' already exists. Creating a new version of this model...
2025/07/05 06:02:30 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: IRIS-classifier-dt-hyperparameter, version 12


🏃 View run grid_run_12 at: http://127.0.0.1:8100/#/experiments/384316239563522829/runs/b7420914d4654a18978d8cfac1ad7d69
🧪 View experiment at: http://127.0.0.1:8100/#/experiments/384316239563522829
✅ Best Params: {'max_depth': 2, 'min_samples_leaf': 1, 'min_samples_split': 2}
✅ Best CV Score: 0.9333333333333332


Created version '12' of model 'IRIS-classifier-dt-hyperparameter'.
